In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# import arya

- P16: 10.26093/cds/vizier.22250024. Table 12. 
- R18 is available at DOI: 10.11570/18.0002

In [ ]:
path = "/astro/dboyea/source/VICE/vice/yields/agb/pignatari16/raw/"
datadir = "../../data/"

In [ ]:
molmass = pd.read_csv(datadir + "nugrid/mesa_atomic_weights.txt", sep="\t", comment = "#")
molmass.element = molmass.element.str.title()
molmass["atomic_number"] = np.int64(np.arange(1, len(molmass)+1))

molmass.set_index("element", inplace=True)

In [ ]:
solar_Z = pd.read_csv(datadir + "nugrid/grevesse_noels_1993.txt", comment="!", sep="\t")
solar_Z.element = solar_Z.element.str.strip()

solar_Z.set_index("element", inplace=True)

In [ ]:
solar_Z["atomic_weight"] = [molmass.atomic_weight[ele] for ele, r in solar_Z.iterrows()]

In [ ]:
Ysun = 0.2485
Zsun = 0.0179

Y_H = 1 - (Zsun + Ysun) / molmass.atomic_weight["H"]
print(Y_H)
Y_i = Y_H * 10**(solar_Z.A - 12)
solar_Z["Z"] = Y_i * solar_Z.atomic_weight

print(sum(solar_Z.Z[2:]) - Zsun)
solar_Z

In [ ]:
solar_Z.Z / np.sum(solar_Z.Z)

In [ ]:
b19_raw = pd.read_csv(path + "/B19_2.txt", sep="\s+", skiprows=2)

In [ ]:
b19_raw["element"] = b19_raw.Isotope.str.extract(r"(\w+)-")

In [ ]:
b19_ele = b19_raw.groupby("element").sum()
b19_ele["element"] = b19_ele.index

In [ ]:
b19_ele.loc["C"]

In [ ]:
np.sum(b19_raw[b19_raw.element == "C"], axis=0)

In [ ]:
b19_raw[b19_raw.element == "C"]

In [ ]:
b19_raw.element.unique()

In [ ]:
B19 = b19_ele.melt(id_vars=["element"],  var_name="model", value_name="mass_yield")

B19["Z"] = 0.
B19["M"] = 0.
B19

B19.loc[B19.model == "m2z1m2", "M"] = 2.
B19.loc[B19.model == "m2z1m2", "Z"] = 0.01
B19.loc[B19.model == "m2z1m2", "Mrem"] = 0.632

B19.loc[B19.model == "m3z1m2", "M"] = 3.
B19.loc[B19.model == "m3z1m2", "Z"] = 0.01
B19.loc[B19.model == "m3z1m2", "Mrem"] = 0.661


B19.loc[B19.model == "m2z2m2", "M"] = 2.
B19.loc[B19.model == "m2z2m2", "Z"] = 0.02
B19.loc[B19.model == "m2z2m2", "Mrem"] = 0.646


B19.loc[B19.model == "m3z2m2", "M"] = 3.
B19.loc[B19.model == "m3z2m2", "Z"] = 0.02
B19.loc[B19.model == "m3z2m2", "Mrem"] = 0.656	

B19.loc[B19.model == "m2z3m2", "M"] = 2.
B19.loc[B19.model == "m2z3m2", "Z"] = 0.03
B19.loc[B19.model == "m2z3m2", "Mrem"] = 0.643


B19.loc[B19.model == "m3z3m2", "M"] = 3.
B19.loc[B19.model == "m3z3m2", "Z"] = 0.03
B19.loc[B19.model == "m3z3m2", "Mrem"] = 0.650

B19[B19.element == "K"]

In [ ]:
elements = "H, He, Li, B, C, N, O, F, Ne, Na, Mg, Al, Si, P, Pb, S, Cl, Ar, K, Ca, Sc, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn, Ga, Ge, As, Se, Br, Kr, Rb, Sr, Y, Zr, Nb, Mo, Ru, Rh, Pd, Ag, Cd, In, Sn, Sb, Te, I, Xe, Cs, Ba, La, Ce, Pr, Nd, Sm, Eu, Gd, Tb, Dy, Ho, Er, Tm, Yb, Lu, Hf, Ta, W, Re, Os, Ir, Pt, Au, Hg, Tl, Bi".split(", ")

In [ ]:
B19 = B19[np.isin(B19.element, elements)]
B19 = B19[B19.M > 0]


B19["Zini"] = [solar_Z.Z[r.element] * r.Z / Zsun for (i, r) in B19.iterrows()]

B19["OP"] = B19.mass_yield / ((B19.M - B19.Mrem) * B19.Zini)
B19_indexed = B19.set_index(["M", "Z", "element"])


B19[B19.element == "K"]


# Comparisons

In [ ]:
def print_properties(M, Z):
    print("B19")
    print("Z=", Z)
    print("M=", M)
    print("Mrem", B19_indexed.Mrem[M, Z, "H"])
    print("model ", B19_indexed.model[M, Z, "H"])
    df = B19_indexed.Zini[M, Z, :]
    Z1 = np.sum(df[~np.isin(df.index, ["H", "He"])])
    print(f"Z = {Z1:0.4f}")

In [ ]:
for M in [2., 3.]:
    plt.figure(figsize=(6, 3))
    for i, Z in enumerate([0.01, 0.02, 0.03]):    
        x  = molmass.loc[B19_indexed.loc[M, Z, :].index.values].atomic_number
        y = B19_indexed.loc[M, Z, :].OP
        model = B19_indexed.model[M, Z, "C"]

        plt.scatter(x, y, 
                    label=f"{model}", s=1, color=["red", "green", "blue"][i],
                    marker = ["s", "^", "o"][i],
        )       
        print_properties(M, Z)
        plt.xlabel("atomic number")
        plt.ylabel("overproduction factor")
        plt.axhline(1, color="black", )
        
        plt.legend()
        plt.title(f"M={M}")
        plt.yscale("log")
        plt.ylim(0.3)
        
    plt.show()